In [163]:
import pandas as pd
import numpy as np
file = "Resources/purchase_data.csv"

purchase_data = pd.read_csv(file)
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

In [164]:
# Player Count

total_players = len(purchase_data["SN"].unique())

pd.DataFrame([{"Total Players":total_players}])

,Total Players
0,576


In [165]:
# Purchasing Analysis (Total)

total_items = len(purchase_data["Item ID"].unique())
avg_price = (purchase_data["Price"].sum())/(len(purchase_data["Price"]))
num_purchases = len(purchase_data["Purchase ID"])
total_revenue = purchase_data["Price"].sum()

pd.DataFrame([{
    "Number of Unique Items":total_items,
    "Average Price":"${:.2f}".format(avg_price),
    "Number of Purchases":num_purchases,
    "Total Revenue":"${:,.2f}".format(total_revenue)
}])

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [166]:
# Gender Demographics

unique_player_ct = purchase_data.drop_duplicates(subset=["SN"])
gender_count = unique_player_ct["Gender"].value_counts()
pct_players = (gender_count/total_players)*100

pd.DataFrame({
    "Total Count":gender_count,
    "Percentage of Players":pct_players.map("{:.2f}%".format)
})

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [167]:
# Gender Purchase Analysis

purchase_count = purchase_data["Gender"].value_counts()
avg_purchase = purchase_data.groupby("Gender")["Price"].mean()
total_purchase = purchase_data.groupby("Gender")["Price"].sum()
avg_per_person = (total_purchase)/(gender_count)

pd.DataFrame({
    "Purchase Count":purchase_count,
    "Average Purchase Price":avg_purchase.map("${:.2f}".format),
    "Total Purchase Value":total_purchase.map("${:,.2f}".format),
    "Avg Total Purchase per Person":avg_per_person.map("${:.2f}".format)
})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [168]:
# Age Demographics

age_ranges = [0, 9, 14, 19, 24, 29, 34, 39, purchase_data["Age"].max()]
age_ind_values = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

age_counts = unique_player_ct.groupby(pd.cut(unique_player_ct["Age"], age_ranges)).count()
age_pct = (age_counts["Age"]/total_players)*100

age_demo_df = pd.DataFrame({
    "Total Count":age_counts["Age"],
    "Percentage of Players":age_pct.map("{:.2f}%".format)
})

age_demo_df.set_index([age_ind_values])

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [169]:
# Age Purchasing Analysis

purchase_group = purchase_data.groupby(pd.cut(purchase_data["Age"], age_ranges))

purch_count = purchase_group["Age"].count()
avg_purch_price = purchase_group["Price"].mean()
total_purch_val = purchase_group["Price"].sum()
avg_total_per_person = total_purch_val/age_counts["Age"]

age_purch_df = pd.DataFrame({
    "Purchase Count":purch_count,
    "Average Purchase Price":avg_purch_price.map("${:,.2f}".format),
    "Total Purchase Value":total_purch_val.map("${:,.2f}".format),
    "Avg Total Prchase per Person":avg_total_per_person.map("${:,.2f}".format)
})

age_purch_df.set_index([age_ind_values])

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Prchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [240]:
# Top Spenders

sn_group = purchase_data.groupby("SN")

sn_purch_count = sn_group["Price"].count()
sn_total_purch_val = sn_group["Price"].sum()
sn_avg_purch_price = sn_total_purch_val/sn_purch_count

top_spender_df = pd.DataFrame({
    "Purchase Count":sn_purch_count,
    "Average Purchase Price":sn_avg_purch_price,
    "Total Purchase Value":sn_total_purch_val,
})

# Formatting for currency. Doing so in lines 11 & 12 causes issues when sorting.
top_spender_df = top_spender_df.sort_values(by="Total Purchase Value", ascending=False).head()
top_spender_df["Average Purchase Price"] = top_spender_df["Average Purchase Price"].map("${:.2f}".format)
top_spender_df["Total Purchase Value"] = top_spender_df["Total Purchase Value"].map("${:.2f}".format)

top_spender_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [241]:
# Most Popular Items

item_group = purchase_data.groupby(["Item ID", "Item Name"])
                                                               #^
item_purch_count = item_group["Item ID"].count()               #|
item_price = item_group["Price"].mean()   #placeholder, need this value
item_total_val = item_group["Price"].sum()

items_df = pd.DataFrame({
    "Purchase Count":item_purch_count,
    "Item Price":item_price,
    "Total Purchase Value":item_total_val
})

items_count = items_df.sort_values(by="Purchase Count", ascending=False).head()
items_count["Item Price"] = items_count["Item Price"].map("${:.2f}".format)
items_count["Total Purchase Value"] = items_count["Total Purchase Value"].map("${:.2f}".format)

items_count

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [243]:
# Most Profitable Items

total_value = items_df.sort_values(by="Total Purchase Value", ascending=False).head()
total_value["Item Price"] = total_value["Item Price"].map("${:.2f}".format)
total_value["Total Purchase Value"] = total_value["Total Purchase Value"].map("${:.2f}".format)

total_value

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
